# **Importations datasets**

In [2]:
import pandas as pd
#! === IMPORT ===


#* Les datasets importants
df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')

#* Les datasets intéressants
df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_title_crew = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_name_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')

MemoryError: Unable to allocate 86.0 MiB for an array with shape (11270719,) and data type object

# **Initialisation Principal**

In [1]:

#! === SETUP ===
#? title_basics

#* Garder uniquement les films
df_title_basics_clean = df_title_basics[df_title_basics['titleType'] == 'movie']

#* Supprimer les lignes avec des données manquantes essentielles
df_title_basics_clean = df_title_basics_clean.dropna(subset=['startYear', 'genres'])

#* Convertir startYear en entier et filtrer les années valides
df_title_basics_clean['startYear'] = df_title_basics_clean['startYear'].astype(int)
df_title_basics_clean = df_title_basics_clean[(df_title_basics_clean['startYear'] >= 1970) & (df_title_basics_clean['startYear'] <= 2024)]

#* Garder les colonnes essentiels
df_title_basics_clean = df_title_basics_clean[['tconst', 'primaryTitle', 'startYear', 'genres']]



#! === SETUP ===
#? title_rating

#* Supprimer les films avec peu de votes
df_title_ratings_clean = df_title_ratings[df_title_ratings['numVotes'] > 1000]



#! === SETUP ===
#? title_akas

#* Garder uniquement les titres de film français
df_title_akas_clean = df_title_akas[df_title_akas['region'] == 'FR']

#* Garder les colonnes essentiels
df_title_akas_clean = df_title_akas_clean[['titleId', 'title']]



#! === SETUP ===
#? title_crew & name_basics

#* Supprimer les lignes avec des valeurs manquantes dans 'directors'
df_title_crew_clean = df_title_crew.dropna(subset=['directors'])

#* Remplir les valeurs manquantes avec un indicateur, par exemple 'Unknown'
df_title_crew_clean = df_title_crew_clean.fillna({'directors': 'Unknown'})

#* Transformer la colonne 'directors' en une liste
df_title_crew_clean['directors'] = df_title_crew_clean['directors'].str.split(',')

#* Exploser la colonne 'directors' en lignes individuelles
df_title_crew_clean = df_title_crew_clean.explode('directors')

#* Fusionner les 'directors' avec 'name_basics' pour obtenir les 'primaryName'
title_crew_with_directors = df_title_crew_clean.merge(df_name_basics[['nconst', 'primaryName']], 
                                                     how='left', 
                                                     left_on='directors', 
                                                     right_on='nconst')

#* Regrouper les résultats par 'tconst' pour remettre les réalisateurs sous forme de liste
title_crew_with_directors = title_crew_with_directors.groupby('tconst')['primaryName'].apply(list).reset_index()

NameError: name 'df_title_basics' is not defined

# **Renommer les colonnes**

In [3]:

#! === RENAMEUP ===


#* Renommer les colonnes de title_basics
df_title_basics_clean.rename(columns={'tconst': 'ID',
                                      'primaryTitle': 'Titre',
                                      'startYear': 'Année',
                                      'genres':  'Genres'}, inplace=True)

#* Renommer les colonnes de title_ratings
df_title_ratings_clean.rename(columns={'tconst': 'ID',
                                       'averageRating': 'Note',
                                       'numVotes': 'Votes'}, inplace=True)

#* Renommer les colonnes de title_akas
df_title_akas_clean.rename(columns={'titleId': 'ID', 
                                    'title': 'Titre Français'}, inplace=True)

#* Renommer les colonnes de title_crew & name_basics
title_crew_with_directors.rename(columns={'primaryName': 'Nom du/des Réalisateur(s)',
                                          'tconst': 'ID'}, inplace=True)

C:\Users\koke7\AppData\Local\Temp\ipykernel_27592\1864161527.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_title_ratings_clean.rename(columns={'tconst': 'ID',


# **Fusion Principal**

In [4]:

#! === MERGEUP ===


#* Merge de title_basics & title_ratings
df_merged_v1 = df_title_basics_clean.merge(df_title_ratings_clean, on="ID")

#* Merge de df_merged_v1 & title_ratings
df_merged_v2 = df_merged_v1.merge(df_title_akas_clean, on='ID', how='left')

#* Merge de title_basics & title_ratings
df_merged_v3 = df_merged_v2.merge(title_crew_with_directors, on='ID', how='left')


# **Exportation**

In [ ]:

#! === EXPORT ===


df_merged_v3.to_csv("../data/raw/df_movie.csv", index=False)

# **Importation**

In [ ]:
import pandas as pd

#! === IMPORT ===


file_path = "../data/raw/df_movie.csv"

df_movie = pd.read_csv(file_path)

In [2]:
df_movie

,ID,Titre,Année,Genres,Note,Votes,Titre Français,Nom du/des Réalisateur(s)
0,tt0035423,Kate & Leopold,2001,"Comedy,Fantasy,Romance",6.4,90632,Kate et Léopold,['James Mangold']
1,tt0038687,Let There Be Light,1980,"Documentary,War",7.4,2063,Que la lumière soit,['John Huston']
2,tt0054724,I Eat Your Skin,1971,Horror,3.6,1717,NaN,['Del Tenney']
3,tt0061592,Doomsday Machine,1976,Sci-Fi,2.6,1421,NaN,"['Harry Hope', 'Lee Sholem', 'Herbert J. Leder']"
4,tt0062690,The Awakening of the Beast,1970,"Drama,Horror",5.9,1339,L'éveil de la bête,['José Mojica Marins']
...,...,...,...,...,...,...,...,...
41557,tt9907782,The Cursed,2021,"Fantasy,Horror,Mystery",6.2,18984,The Cursed,['Sean Ellis']
41558,tt9908390,Le lion,2020,Comedy,5.5,1457,Le lion,['Ludovic Colbeau-Justin']
41559,tt9911196,The Marriage Escape,2020,"Comedy,Drama",7.4,3391,De Beentjes van Sint-Hildegard,['Johan Nijenhuis']
41560,tt9916270,Il talento del calabrone,2020,Thriller,5.8,1492,NaN,['Giacomo Cimini']


# **Ajout de données supplémentaires**

In [ ]:

#! === ADDUP ===


#* Ajout de la colonne popularité
def categorize_votes(votes):
    if votes < 25000:
        return "Connu"
    elif 25000 <= votes < 100000:
        return "Populaire"
    elif 100000 <= votes < 500000:
        return "Très populaire"
    else:
        return "Blockbuster"

df_movie['Popularité'] = df_movie['Votes'].apply(categorize_votes)

#* Ajout de la colonne décennie
df_movie['Décennie'] = (df_movie['Année'].astype(int) // 10) * 10

#* Repositionner les colonnes
df_movie = df_movie[['ID', 'Nom du/des Réalisateur(s)', 'Titre', 'Titre Français', 'Année', 'Décennie', 'Genres', 'Note', 'Votes', 'Popularité']]

# **Processus de nettoyage**

In [4]:

#! === CLEANUP V1 ===


#* Supprimer les doublons
df_movie_cleaned_v1 = df_movie.drop_duplicates(subset='ID')

#* Supprimer les films avec moins de 15 000 votes
df_movie_cleaned_v1 = df_movie_cleaned_v1[df_movie_cleaned_v1['Votes'] > 15000]

#* Remplacer les valeurs manquantes dans 'Titre Français' par les valeurs de 'Titre'
df_movie_cleaned_v1['Titre Français'] = df_movie_cleaned_v1['Titre Français'].fillna(df_movie_cleaned_v1['Titre'])

In [5]:

#! === CLEANUP V2 ===


#* Supprimer les films avec une note inférieur à 5
df_movie_cleaned_v2 = df_movie_cleaned_v1[df_movie_cleaned_v1['Note'] > 5]

#* Supprimer les films avec une note inférieur à 8 et sortis avant 2000
df_movie_cleaned_v2 = df_movie_cleaned_v2[~((df_movie_cleaned_v2['Note'] < 8) & (df_movie_cleaned_v2['Année'] < 2000))]

# **Exportation**

In [6]:

#! === EXPORT ===

df_movie_cleaned_v2.to_csv("../data/processed/df_movie_cleaned.csv", index=False)